<a href="https://colab.research.google.com/github/swarris/simulitis/blob/master/Simulitis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulitis verspreiding
In dit project gaan we de verspreiding van een denkbeeldig virus simuleren. Dit virus, *simulitis* is overdraagbaar van mens tot mens en kan dodelijk zijn. De mate waarin het virus overdraagbaar is en hoe dodelijk het is kan per simulatie aangepast worden. Deze simulatie is geschreven in de programmeertaal Python. Deze simulaties zijn **illustratief** en om **programmeren in Python te leren**. De rekenmodelen zijn **niet wetenschappelijk**! 
## Laten we een wereld maken
Onze wereld is vierkant en is *X* lang en *Y* breed. Met de infectie afstand kunnen aangeven hoe dicht iemand op een besmet persoon moet staan om ook ziek te worden.

In [0]:
class Wereld:

  def __init__(self, X, Y, aantalPersonen, infectieAfstand):
    self.X = X
    self.Y = Y
    self.aantalPersonen = aantalPersonen
    self.infectieAfstand = infectieAfstand
    self.populatie = []

  def maak(self):
    """ 
    Deze methode maakt een populatie aan met #aantalPersonen en start met
    1 ziek persoon.
    """
    self.populatie = []
    for i in range(self.aantalPersonen):
      self.populatie.append(Persoon(100*random.random(), self, self.X * random.random(),self.Y*random.random()))
    # maak er 1 ziek
    self.populatie[0].geinfecteerd = True


## Nu moeten we een Persoon definieren
Een persoon heeft een leeftijd en startlocatie in de wereld. Ook kan de persoon door de wereld lopen.

In [0]:
import math
import random

class Persoon:
  def __init__(self, leeftijd, wereld, startX, startY):  
    """
    Een persoon heeft een leeftijd, een wereld om in te leven en
    een start positie in deze wereld. Hij/zij beweegt in een willekeurige 
    richting. Een persoon is niet geinfecteerd als hij aangemaakt wordt.
    """
    self.leeftijd = leeftijd
    self.wereld = wereld
    self.X = startX
    self.Y = startY
    self.looprichting = random.random()*math.pi*2.0
    self.geinfecteerd = False

    
  def neemStap(self):
    """
    Deze methode zorgt er voor de persoon een stap maakt in de wereld.
    """

    # verander van richting?
    if random.random() > 0.95:
      self.looprichting = random.random()*math.pi*2.0
    # neem een stap
    self.X = self.X + math.cos(self.looprichting)
    self.Y = self.Y + math.sin(self.looprichting)
    # je mag niet de wereld uit lopen:
    if self.X > self.wereld.X:
      self.X = 0
    elif self.X < 0:
      self.X = self.wereld.X
    if self.Y > self.wereld.Y:
      self.Y = 0
    elif self.Y < 0:
      self.Y = self.wereld.Y

        
  def huidigeLocatie(self):
    """ Waar ben ik nu? """
    return [self.X, self.Y]

  def infecteer(self):
    """ 
    Doorloop alle mensen in de wereld: ben je dichtbij genoeg, 
    dan kan ik je ziek maken (als ik zelf ziek ben). Geeft het aantal 
    besmette mensen terug.
    """
    geinfecteerden = 0
    if self.geinfecteerd:
      for p in self.wereld.populatie:
        if not p.geinfecteerd and math.sqrt((p.X-self.X)**2 + (p.Y-self.Y)**2) <= self.wereld.infectieAfstand :
          p.geinfecteerd = True
          geinfecteerden += 1
    return geinfecteerden


## De mensen op de wereld
Nu we een wereld en personen kunnen maken, gaan we die samenvoegen. Met de volgende variabelen kun je aangeven hoe groot de wereld moet worden en hoeveel mensen er rondlopen. Even een tip: hoe groter de wereld en/of hoe meer mensen hoe langer het duurt voordat straks de simulatie klaar is.

In [0]:
wereldGrootte = 200
aantalPersonen = 200
infectieAfstand = 3.0
tijdstappenInDeAnimatie = 500

Nu kunnen we de wereld gaan aanmaken. We starten met 1 ziek persooon.

In [0]:
wereld = Wereld(wereldGrootte,wereldGrootte, aantalPersonen, infectieAfstand)


Het volgende stuk code is nodig om de aninamatie te maken. De personen lopen door de wereld en kunnen mensen ziek maken.

In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

class Animatie:
  def __init__(self, wereld, tijdstappenInDeAnimatie):
    """
    Definitie van het plot venster.
    """
    self.wereld = wereld
    self.tijdstappenInDeAnimatie = tijdstappenInDeAnimatie
    self.fig, (self.ax, self.ax2) = plt.subplots(2,1,figsize=(8,12))
    self.ax.set_xlim(( 0, wereld.X))
    self.ax.set_ylim((0, wereld.Y))


    self.ax2.set_xlim(( 0, tijdstappenInDeAnimatie))
    self.ax2.set_ylim((0, 100))



    # We houden zieken en gezonden bij
    self.gezond, = self.ax.plot([], [], "bo", lw=2)
    self.ziek, = self.ax.plot([], [], "ro", lw=2)
    self.totaalZiek, = self.ax2.plot([], [], "ro", lw=2)
    self.legend = self.ax2.legend((self.gezond, self.ziek), 
                                  ("Gezond", "Ziek"), 
                                   loc="lower left",bbox_to_anchor= (0.0, 1.01), borderaxespad=0., ncol=2)
    self.totaalZiekStatus = []

  def init(self):
    """ Leegmaken van de data voor de grafiek """
    self.gezond.set_data([], [])
    self.ziek.set_data([], [])
    self.totaalZiek.set_data([],[])
    return (self.gezond,self.ziek, self.totaalZiek)

  def animeer(self, i):
    """ 
    Maken van de animatie. Laat de personen in de wereld rondlopen en
    elkaar infecteren. 
    """
    gezondX = []
    gezondY = []
    ziekX = []
    ziekY = []
    self.totaalZiekStatus.append(0.0)

    # Loop
    for p in self.wereld.populatie:
      p.neemStap()
    # Maak ziek
    for p in self.wereld.populatie:
      p.infecteer()
    
    # Bekijk huidige situatie
    for p in self.wereld.populatie:
      locatie = p.huidigeLocatie()
      if p.geinfecteerd:        
        ziekX.append(locatie[0])
        ziekY.append(locatie[1])
        self.totaalZiekStatus[-1] += 1.0
      else:
        gezondX.append(locatie[0])
        gezondY.append(locatie[1])
    
    self.gezond.set_data(gezondX, gezondY)
    self.ziek.set_data(ziekX, ziekY)
    self.totaalZiekStatus[-1] = self.totaalZiekStatus[-1] /len(wereld.populatie) * 100.0
    self.totaalZiek.set_data(list(range(0,i+1)), self.totaalZiekStatus)
    return (self.gezond, self.ziek, self.totaalZiek)

  def maakVenster(self):
    """ Initialisatie van het animeer functie """
    self.venster = animation.FuncAnimation(self.fig, self.animeer, init_func=self.init,
                               frames=self.tijdstappenInDeAnimatie, interval=1, blit=True)
 

## De animatie
Het kan even duren voordat de animatie klaar is: het hangt af van hoe groot de wereld is, hoeveel mensen er rond lopen en hoeveel frames er gemaakt moeten worden.
Verander *to_jshtml* in *to_html5_video* om er een video van te maken. 

In [0]:
%%capture
animatie = Animatie(wereld, tijdstappenInDeAnimatie)

In [0]:
wereld.maak()
animatie.maakVenster()
HTML(animatie.venster.to_jshtml())

In [0]:
wereld.maak()
animatie.maakVenster()
HTML(animatie.venster.to_html5_video())

## Immuniteit
Mensen worden na verloop van tijd immuum.

In [0]:
%%capture
class MetAfweer(Persoon):
  def __init__(self, X, Y, aantalPersonen, infectieAfstand):
    super().__init__(X, Y, aantalPersonen, infectieAfstand)
    self.immuum = False
    self.gemaaktePassen = 0
  
  def wordtBeter(self, aantalPassen):
    """ Een persoon wordt beter en daarmee immuum na verloop van tijd. """
    if self.gemaaktePassen >= aantalPassen and self.geinfecteerd:
      self.geinfecteerd = False
      self.immuum = True

  def infecteer(self):
    """ 
    Doorloop alle mensen in de wereld: ben je dichtbij genoeg, 
    dan kan ik je ziek maken (als ik zelf ziek ben). Geeft het aantal 
    besmette mensen terug.
    """
    geinfecteerden = 0
    if self.geinfecteerd:
      for p in self.wereld.populatie:
        if not p.geinfecteerd and not p.immuum and math.sqrt((p.X-self.X)**2 + (p.Y-self.Y)**2) <= self.wereld.infectieAfstand :
          p.geinfecteerd = True
          geinfecteerden += 1
    return geinfecteerden

  def neemStap(self):
    """ Hou nu ook bij hoeveel stappen er gezet zijn. """
    super().neemStap()
    if self.geinfecteerd:
      self.gemaaktePassen += 1    


class BetereWereld(Wereld):
  """ De BetereWereld bestaat uit personen die beter en immuum kunnen worden """
  def __init__(self, X, Y, aantalPersonen, infectieAfstand):
    super().__init__(X, Y, aantalPersonen, infectieAfstand)

  def maak(self):
    self.populatie = []
    for i in range(self.aantalPersonen):
      self.populatie.append(MetAfweer(100*random.random(), self, self.X * random.random(),self.Y*random.random()))
    self.populatie[0].geinfecteerd = True

class BetereAnimatie(Animatie):
  def __init__(self, wereld, tijdstappenInDeAnimatie, aantalPassen):
    super().__init__(wereld, tijdstappenInDeAnimatie)
    self.immuum, = self.ax.plot([], [], "go", lw=2)
    self.totaalImmuum, = self.ax2.plot([], [], "go", lw=2)
    self.legend.remove()
    self.legend = self.ax2.legend((self.gezond, self.ziek, self.immuum), 
                                  ("Gezond", "Ziek", "Beter & immuum"),
                                  loc="lower left",bbox_to_anchor= (0.0, 1.01), borderaxespad=0., ncol=3)
    self.totaalImmuumStatus = []
    self.aantalPassen = aantalPassen
    

  def init(self):
    super().init()
    self.immuum.set_data([], [])
    self.totaalImmuum.set_data([], [])
    return (self.gezond,self.ziek, self.immuum, self.totaalImmuum)

  def animeer(self, i):
    gezondX = []
    gezondY = []
    ziekX = []
    ziekY = []
    immuumX = []
    immuumY = []
    self.totaalZiekStatus.append(0.0)
    self.totaalImmuumStatus.append(0.0)
    
    for p in self.wereld.populatie:
      p.neemStap()
    for p in self.wereld.populatie:
      p.infecteer()
    for p in self.wereld.populatie:
      p.wordtBeter(self.aantalPassen)

    
    for p in self.wereld.populatie:
      locatie = p.huidigeLocatie()
      if p.geinfecteerd:        
        ziekX.append(locatie[0])
        ziekY.append(locatie[1])
        self.totaalZiekStatus[-1] += 1.0
      elif p.immuum:
        immuumX.append(locatie[0])
        immuumY.append(locatie[1])
        self.totaalImmuumStatus[-1] += 1.0
      else:
        gezondX.append(locatie[0])
        gezondY.append(locatie[1])

    self.gezond.set_data(gezondX, gezondY)
    self.ziek.set_data(ziekX, ziekY)
    self.totaalZiekStatus[-1] = self.totaalZiekStatus[-1] /len(wereld.populatie) * 100.0
    self.totaalZiek.set_data(list(range(0,i+1)), self.totaalZiekStatus)    
    self.totaalImmuumStatus[-1] = self.totaalImmuumStatus[-1] /len(wereld.populatie) * 100.0
    self.totaalImmuum.set_data(list(range(0,i+1)), self.totaalImmuumStatus)
    self.immuum.set_data(immuumX, immuumY)

    return (self.gezond, self.ziek, self.immuum, self.totaalZiek, self.totaalImmuum)



In [0]:
%%capture
aantalPassen = 150
betereWereld = BetereWereld(wereldGrootte,wereldGrootte, aantalPersonen, infectieAfstand)
betereAnimatie = BetereAnimatie(betereWereld, tijdstappenInDeAnimatie, aantalPassen) 


In [0]:
betereWereld.maak()
betereAnimatie.maakVenster()
HTML(betereAnimatie.venster.to_jshtml())

## Overlijden
In de volgende simulatie nemen we ook mee dat mensen kunnen komen te overlijden. Bij elke stap is de kans dat iemand komt te overlijden evenredig met zijn leeftijd: hoe ouder, hoe groter de kans.

In [0]:
class Mens(MetAfweer):
  def __init__(self, X, Y, aantalPersonen, infectieAfstand):
    super().__init__(X, Y, aantalPersonen, infectieAfstand)
    self.levend = True

  def neemStap(self):
    """ Hou nu ook bij hoeveel stappen er gezet zijn. """
    if self.levend:
      super().neemStap()
      if self.geinfecteerd and self.gemaaktePassen % 50 == 0:
        self.levend = (random.random() * 100.0) > self.leeftijd
    if not self.levend:
      self.geinfecteerd = False

class EchteWereld(BetereWereld):
  """ De EchteWereld bestaat uit personen die ook kunnen sterven """
  def __init__(self, X, Y, aantalPersonen, infectieAfstand):
    super().__init__(X, Y, aantalPersonen, infectieAfstand)

  def maak(self):
    self.populatie = []
    for i in range(self.aantalPersonen):
      self.populatie.append(Mens(100*random.random(), self, self.X * random.random(),self.Y*random.random()))
    self.populatie[0].geinfecteerd = True

class EchteAnimatie(BetereAnimatie):
  def __init__(self, wereld, tijdstappenInDeAnimatie, aantalPassen):
    super().__init__(wereld, tijdstappenInDeAnimatie, aantalPassen)
    self.dood, = self.ax.plot([], [], "k+", lw=2)
    self.totaalDoodStatus = []
    self.gemiddeldeLeeftijdStatus = []
    self.totaalDood, = self.ax2.plot([], [], "k+", lw=2)
    self.gemiddeldeLeeftijd, = self.ax2.plot([], [], "b-", lw=2)
    self.legend.remove()
    self.legend = self.ax2.legend((self.gezond, self.ziek, self.immuum, self.totaalDood, self.gemiddeldeLeeftijd), 
                                  ("Gezond", "Ziek", "Beter & immuum", "Dood", "Gemiddelde leeftijd"), 
                                  loc="lower left",bbox_to_anchor= (0.0, 1.01), borderaxespad=0., ncol=5)


  def init(self):
    super().init()
    self.dood.set_data([], [])
    self.totaalDood.set_data([],[])
    self.gemiddeldeLeeftijd.set_data([],[])
    return (self.gezond,self.ziek, self.immuum, self.dood, self.totaalZiek, self.totaalImmuum, self.totaalDood, self.gemiddeldeLeeftijd)

  def animeer(self, i):
    gezondX = []
    gezondY = []
    ziekX = []
    ziekY = []
    immuumX = []
    immuumY = []
    doodX = []
    doodY = []

    self.totaalZiekStatus.append(0.0)
    self.totaalImmuumStatus.append(0.0)
    self.totaalDoodStatus.append(0.0)
    self.gemiddeldeLeeftijdStatus.append(0.0)

    
    for p in self.wereld.populatie:
      p.neemStap()
    for p in self.wereld.populatie:
      p.infecteer()
    for p in self.wereld.populatie:
      p.wordtBeter(self.aantalPassen)

    
    for p in self.wereld.populatie:
      locatie = p.huidigeLocatie()
      if not p.levend:
        doodX.append(locatie[0])
        doodY.append(locatie[1])
        self.totaalDoodStatus[-1] += 1.0
      elif p.geinfecteerd:        
        ziekX.append(locatie[0])
        ziekY.append(locatie[1])
        self.totaalZiekStatus[-1] += 1.0
      elif p.immuum:
        immuumX.append(locatie[0])
        immuumY.append(locatie[1])
        self.totaalImmuumStatus[-1] += 1.0
      else:
        gezondX.append(locatie[0])
        gezondY.append(locatie[1])
      if p.levend:
        self.gemiddeldeLeeftijdStatus[-1] += p.leeftijd

    self.gezond.set_data(gezondX, gezondY)
    self.ziek.set_data(ziekX, ziekY)
    self.immuum.set_data(immuumX, immuumY)
    self.dood.set_data(doodX, doodY)

    self.totaalZiekStatus[-1] = self.totaalZiekStatus[-1] /len(wereld.populatie) * 100.0
    self.totaalZiek.set_data(list(range(0,i+1)), self.totaalZiekStatus)    
    self.totaalImmuumStatus[-1] = self.totaalImmuumStatus[-1] /len(wereld.populatie) * 100.0
    self.totaalImmuum.set_data(list(range(0,i+1)), self.totaalImmuumStatus)

    self.gemiddeldeLeeftijdStatus[-1] = self.gemiddeldeLeeftijdStatus[-1] /(len(wereld.populatie)-self.totaalDoodStatus[-1])
    self.gemiddeldeLeeftijd.set_data(list(range(0,i+1)), self.gemiddeldeLeeftijdStatus)
    self.totaalDoodStatus[-1] = self.totaalDoodStatus[-1] /len(wereld.populatie) * 100.0
    self.totaalDood.set_data(list(range(0,i+1)), self.totaalDoodStatus)    

    return (self.gezond, self.ziek, self.immuum, self.totaalZiek, self.totaalImmuum, self.dood, self.totaalDood, self.gemiddeldeLeeftijd)

In [0]:
%%capture
echteWereld = EchteWereld(wereldGrootte,wereldGrootte, aantalPersonen, infectieAfstand)
echteAnimatie = EchteAnimatie(echteWereld, tijdstappenInDeAnimatie, aantalPassen)

In [0]:
echteWereld.maak()
echteAnimatie.maakVenster()
HTML(echteAnimatie.venster.to_html5_video())